<h2>Regression Models with Keras</h2>

### Download and Clean Dataset


Let's start by importing the <em>pandas</em> and the Numpy libraries.


In [2]:
import pandas as pd
import numpy as np

Let's download the data and read it into a <em>pandas</em> dataframe.


In [3]:
data_location = "https://cocl.us/concrete_data"
concrete_data = pd.read_csv(data_location)
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


Let's check how many data points we have.


In [4]:
concrete_data.shape

(1030, 9)

Let's check the dataset for any missing values.


In [5]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [6]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.


#### Split data into predictors and target


The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.


In [7]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

lets normalize the predictors

In [8]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


Let's do a quick sanity check of the predictors and the target dataframes.


In [9]:
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [10]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Let's save the number of predictors to _n_cols_ since we will need this number when building our network.
NOTE: the instruction says 7 predictors (excluding *age*), but here I just keep all 8 original predictors in.


In [11]:
n_cols = predictors_norm.shape[1] # number of predictors
print(n_cols)

8


<h4>split data into train and test data</h4>

In [12]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=42)

<a id='item32'></a>


## Import Keras


In [13]:
from keras.models import Sequential
from keras.layers import Dense

<a id='item33'></a>


## Build a Neural Network


Let's define a function that defines our regression model for us so that we can conveniently call it to create our model.


In [14]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

The above function create a model that has one hidden layer of 10 hidden units.


<a id="item4"></a>


<a id='item34'></a>


## Train and Test the Network


Next, we will train and test the model at the same time using the _fit_ method. We will leave out 30% of the data for validation and we will train the model for 50 epochs.


In [15]:
# fit the model
def fit_model(model):
    model.fit(X_train, y_train, validation_split=0.3, epochs=100, verbose=0)

<h4>evaluate test dataset</h4>

In [16]:
# evaluate the model
def eval_model(model):
    return model.evaluate(X_test, y_test, verbose=2, return_dict=True)

<h4>repeat fit&evaluate 50 times</h4>

In [17]:
n_run = 50
l_mse = []
for i in range(n_run):
    print("#run: {}".format(i+1))
    model = regression_model()
    fit_model(model)
    l_mse.append(eval_model(model)["loss"])

#run: 1
10/10 - 0s - loss: 139.5779
#run: 2
10/10 - 0s - loss: 132.0110
#run: 3
10/10 - 0s - loss: 138.6401
#run: 4
10/10 - 0s - loss: 123.5791
#run: 5
10/10 - 0s - loss: 135.6141
#run: 6
10/10 - 0s - loss: 129.1459
#run: 7
10/10 - 0s - loss: 133.0677
#run: 8
10/10 - 0s - loss: 134.7373
#run: 9
10/10 - 0s - loss: 132.9442
#run: 10
10/10 - 0s - loss: 133.6735
#run: 11
10/10 - 0s - loss: 107.4911
#run: 12
10/10 - 0s - loss: 129.1607
#run: 13
10/10 - 0s - loss: 128.8388
#run: 14
10/10 - 0s - loss: 129.3867
#run: 15
10/10 - 0s - loss: 131.0011
#run: 16
10/10 - 0s - loss: 110.0579
#run: 17
10/10 - 0s - loss: 127.1711
#run: 18
10/10 - 0s - loss: 131.4173
#run: 19
10/10 - 0s - loss: 125.9457
#run: 20
10/10 - 0s - loss: 136.1791
#run: 21
10/10 - 0s - loss: 129.2661
#run: 22
10/10 - 0s - loss: 134.3369
#run: 23
10/10 - 0s - loss: 137.6073
#run: 24
10/10 - 0s - loss: 130.0929
#run: 25
10/10 - 0s - loss: 133.9344
#run: 26
10/10 - 0s - loss: 124.0893
#run: 27
10/10 - 0s - loss: 133.5922
#run: 28
1

In [19]:
print("MSE mean: {:.3f}, MSE std: {:.3f}".format(np.mean(l_mse), np.std(l_mse)))

MSE mean: 129.080, MSE std: 7.432


## report results

| #re-run | mean of mse | std of mse |
| ------- | ----------- | ---------- |
| 50      | 129.080     | 7.432      |


<h2>Discussion MSE</h2>
comparing with Part B, through increasing number of epoch in Part C:

- mean of MSE improves (Part B: 164.036, Part C: 129.080)

- standard deviation of MSE improves (Part B: 9.954, Part C: 7.432)